**为了方便注解，在代码块之间进行了分割，因此本页代码不能正常运行**  
源代码可查看vanilla_vae.py

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from _type import *
#最原始的变分自动编码器
class VanillaVAE(nn.Module):
    
    def __init__(self,
                 in_channels: int,
                 latent_dim: int,
                 hidden_dims: List = None,
                 **kwargs) -> None:
        super(VanillaVAE, self).__init__()

        self.latent_dim = latent_dim

laten_dim指的是上图中间隐含层的维数
![image.png](./photos/1.webp)

In [ ]:
        modules = []
        if hidden_dims is None:
            hidden_dims = [32, 64, 128, 256, 512]
            #可以指定每个模型隐含层的大小

        # Build Encoder
        for h_dim in hidden_dims:
            modules.append(
                nn.Sequential(
                    nn.Conv2d(in_channels, out_channels=h_dim,
                              kernel_size= 3, stride= 2, padding  = 1),
                    nn.BatchNorm2d(h_dim),
                    nn.LeakyReLU())
            )
            in_channels = h_dim

        self.encoder = nn.Sequential(*modules)
        self.fc_mu = nn.Linear(hidden_dims[-1]*4, latent_dim)
        self.fc_var = nn.Linear(hidden_dims[-1]*4, latent_dim)

使用卷积神经网络来构筑图像编码器  
从Data_loader中取得的向量维度应该是[B,C,H,W],分别对应Batch_Size,图像通道数(彩图就是三通道),图像高度和宽度  
感谢pytorch的封装，我们只需要设定一些简单的参数即可，实际上对于用nn.Sequential堆叠起来的网络模块，只要上一个的out_channels和下一个的in_channels相等就能够运行了。  
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')  
inputs: $(N,C_{in} ,H,W)$  
outputs:$(N,C_{out} ,H_{out},W_{out})$  
为了让隐变量服从高斯分布，在此显式地添加两个全连接层来分别生成高斯分布的均值和方差
![image.png](./photos/1.webp)

In [ ]:
 # Build Decoder
        modules = []

        self.decoder_input = nn.Linear(latent_dim, hidden_dims[-1] * 4)

先用线性层将隐含层z的向量扩展为decoder输入层大小的四倍，再展开为[B x $C_{decoder\_input}$ x 2 x 2]大小的feature map.这主要是为了先扩增特征图大小，方便进行如上图所示的反卷积。虽说1×1的feature map经过padding也可以上采样，但是如果要产生[B x $C_{decoder\_input}$ x 2 x 2]的特征图，恐怕需要C_{decoder\_input}个卷积核，计算效率上不太经济

In [ ]:
        hidden_dims.reverse()

        for i in range(len(hidden_dims) - 1):
            modules.append(
                nn.Sequential(
                    nn.ConvTranspose2d(hidden_dims[i],
                                       hidden_dims[i + 1],
                                       kernel_size=3,
                                       stride = 2,
                                       padding=1,
                                       output_padding=1),
                    nn.BatchNorm2d(hidden_dims[i + 1]),
                    nn.LeakyReLU())
            )



        self.decoder = nn.Sequential(*modules)

        self.final_layer = nn.Sequential(
                            nn.ConvTranspose2d(hidden_dims[-1],
                                               hidden_dims[-1],
                                               kernel_size=3,
                                               stride=2,
                                               padding=1,
                                               output_padding=1),
                            nn.BatchNorm2d(hidden_dims[-1]),
                            nn.LeakyReLU(),
                            nn.Conv2d(hidden_dims[-1], out_channels= 3,
                                      kernel_size= 3, padding= 1),
                            nn.Tanh())

使用转置卷积构筑图像解码器，隐含层的大小顺序与编码器的相反  
转置卷积动态图：
![image.png](./photos/padding_strides_transposed.gif)

In [ ]:
    def encode(self, input: Tensor) -> List[Tensor]:
        """
        Encodes the input by passing through the encoder network
        and returns the latent codes.
        :param input: (Tensor) Input tensor to encoder [N x C x H x W]
        :return: (Tensor) List of latent codes
        """
        result = self.encoder(input)
        result = torch.flatten(result, start_dim=1)

        # Split the result into mu and var components
        # of the latent Gaussian distribution
        mu = self.fc_mu(result)
        log_var = self.fc_var(result)

        return [mu, log_var]

    def decode(self, z: Tensor) -> Tensor:
        """
        Maps the given latent codes
        onto the image space.
        :param z: (Tensor) [B x D]
        :return: (Tensor) [B x C x H x W]
        """
        result = self.decoder_input(z)
        result = result.view(-1, hidden_dims[-1], 2, 2)
        result = self.decoder(result)
        result = self.final_layer(result)
        return result

    def reparameterize(self, mu: Tensor, logvar: Tensor) -> Tensor:
        """
        Reparameterization trick to sample from N(mu, var) from
        N(0,1).
        :param mu: (Tensor) Mean of the latent Gaussian [B x D]
        :param logvar: (Tensor) Standard deviation of the latent Gaussian [B x D]
        :return: (Tensor) [B x D]
        """
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return eps * std + mu

    def forward(self, input: Tensor, **kwargs) -> List[Tensor]:
        mu, log_var = self.encode(input)
        z = self.reparameterize(mu, log_var)
        return  [self.decode(z), input, mu, log_var]

    def loss_function(self,
                      *args,
                      **kwargs) -> dict:
        """
        Computes the VAE loss function.
        KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}
        :param args:
        :param kwargs:
        :return:
        """
        recons = args[0]
        input = args[1]
        mu = args[2]
        log_var = args[3]

        kld_weight = kwargs['M_N'] # Account for the minibatch samples from the dataset
        recons_loss =F.mse_loss(recons, input)


        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var - mu ** 2 - log_var.exp(), dim = 1), dim = 0)

        loss = recons_loss + kld_weight * kld_loss
        return {'loss': loss, 'Reconstruction_Loss':recons_loss, 'KLD':-kld_loss}

损失函数由两个部分组成，一个是图像重构损失，另一个是与标准正态分布的KL散度损失  
generation_loss = mean(square(generated_image - real_image))  
latent_loss = KL-Divergence(latent_variable, unit_gaussian)  
衡量两个高斯分布的KL散度的公式：
$KL(N(\mu, \sigma), N(0, 1)) = \log \frac{1}{\sigma} + \frac{\sigma^2 + \mu^2}{2} - \frac{1}{2}$  
kld_weight对应的是Batch_size/len(datasets)，原论文中就是如此设置的。有人在stackoverchange社区上反映，这个数值太大可能导致重构误差大，过小可能导致采样重建效果不佳，笔者水平有限，暂且不知道这个数值为何如此设置

In [ ]:
    def sample(self,
               num_samples:int,
               current_device: int, **kwargs) -> Tensor:
        """
        Samples from the latent space and return the corresponding
        image space map.
        :param num_samples: (Int) Number of samples
        :param current_device: (Int) Device to run the model
        :return: (Tensor)
        """
        z = torch.randn(num_samples,
                        self.latent_dim)

        z = z.to(current_device)

        samples = self.decode(z)
        return samples

    def generate(self, x: Tensor, **kwargs) -> Tensor:
        """
        Given an input image x, returns the reconstructed image
        :param x: (Tensor) [B x C x H x W]
        :return: (Tensor) [B x C x H x W]
        """

        return self.forward(x)[0]

从正态分布中采样隐层变量，交给解码器进行解码